In [1]:
import numpy as np
import json
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam #SGD, Adadelta, RMSprop
from keras import layers

In [2]:
# Open and Read JSON file's data

file = open(r"shallow-frequencies/shallow-frequencies.json", "r", encoding='utf-8')
js = file.read()
data = json.loads(js)
file.close()

In [3]:
# Get all the proofs
proofs = data["triples"]

types = [row["name"] for row in data["types"]]
terms = [row["name"] for row in data["terms"]]

In [4]:
input_list = []
output_list = []

len_types = len(types)
len_terms = len(terms)

for proof in proofs:
    input_def = [0]*len_types
    output_def = [0]*len_terms
    
    typs = set(proof['types'])
    for typ in typs:
        input_def[types.index(typ)] += proof['types'].count(typ)
    input_list.append(input_def)

    trms = set(proof['terms'])
    for trm in trms:
        output_def[terms.index(trm)] += proof['types'].count(trm)
    output_list.append(output_def)

In [5]:
input_list = np.asarray(input_list)
output_list = np.asarray(output_list)

In [6]:
#no_test = 300 no_test/len(proofs)
x_train, x_test, y_train, y_test = train_test_split(input_list, output_list, test_size=0.15)

In [7]:
model = Sequential()

model.add(Dense(200, input_shape = (len_types,), activation = "softplus"))
model.add(Dropout(0.2))

model.add(Dense(len_terms, activation = "softplus"))
model.add(Dropout(0.05))

model.compile(Adam(learning_rate = 0.002), "categorical_crossentropy", metrics = ["KLDivergence"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               142000    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 3337)              670737    
                                                                 
 dropout_1 (Dropout)         (None, 3337)              0         
                                                                 
Total params: 812,737
Trainable params: 812,737
Non-trainable params: 0
_________________________________________________________________


In [8]:
epoch = 250

### Performance of New model

In [9]:
model.fit(x_train, y_train, epochs = 100, validation_data=(x_test, y_test))

Epoch 1/100
182/182 [==============================] - 2s 8ms/step - loss: 5.1555 - kullback_leibler_divergence: 1.4240 - val_loss: 3.8755 - val_kullback_leibler_divergence: 1.3026
Epoch 2/100
182/182 [==============================] - 2s 8ms/step - loss: 3.9332 - kullback_leibler_divergence: 1.7882 - val_loss: 3.2244 - val_kullback_leibler_divergence: 1.6746
Epoch 3/100
182/182 [==============================] - 1s 8ms/step - loss: 3.3042 - kullback_leibler_divergence: 1.8032 - val_loss: 2.7875 - val_kullback_leibler_divergence: 1.7092
Epoch 4/100
182/182 [==============================] - 2s 8ms/step - loss: 3.0656 - kullback_leibler_divergence: 1.7172 - val_loss: 2.5071 - val_kullback_leibler_divergence: 1.6775
Epoch 5/100
182/182 [==============================] - 1s 8ms/step - loss: 2.6881 - kullback_leibler_divergence: 1.6092 - val_loss: 2.3006 - val_kullback_leibler_divergence: 1.5333
Epoch 6/100
182/182 [==============================] - 2s 8ms/step - loss: 2.6413 - kullback_le

### 

### Performance of Models in verbose_amalgam_predict

In [10]:
import verbose_amalgam_predict as vap

loaded 6823 triples, which use 6823 names
Separated into data_triples: 3984 and test_triples: 463
Model: "factorization_model1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6823)]       0           []                               
                                                                                                  
 repr (Dense)                   (None, 10)           68240       ['input_1[0][0]']                
                                                                                                  
 prob_self (Dense)              (None, 1)            11          ['repr[0][0]']                   
                                                                                                  
 low_rank_out (Dense)           (None, 6823)         75053       ['repr[0][0]'] 

In [11]:
vap.fit(epoch, vap.model1)

Epoch 1/250
63/63 [==============================] - 2s 16ms/step - loss: 6.8544 - kullback_leibler_divergence: 6.8451 - val_loss: 6.6804 - val_kullback_leibler_divergence: 6.6742
Epoch 2/250
63/63 [==============================] - 1s 13ms/step - loss: 6.6230 - kullback_leibler_divergence: 6.6108 - val_loss: 6.4462 - val_kullback_leibler_divergence: 6.4270
Epoch 3/250
63/63 [==============================] - 1s 13ms/step - loss: 6.3972 - kullback_leibler_divergence: 6.3690 - val_loss: 6.2221 - val_kullback_leibler_divergence: 6.1843
Epoch 4/250
63/63 [==============================] - 1s 12ms/step - loss: 6.1821 - kullback_leibler_divergence: 6.1337 - val_loss: 5.9941 - val_kullback_leibler_divergence: 5.9348
Epoch 5/250
63/63 [==============================] - 1s 12ms/step - loss: 5.9591 - kullback_leibler_divergence: 5.8878 - val_loss: 5.7474 - val_kullback_leibler_divergence: 5.6628
Epoch 6/250
63/63 [==============================] - 1s 12ms/step - loss: 5.7161 - kullback_leibler_

In [12]:
vap.fit(epoch, vap.model2)

Epoch 1/250
63/63 [==============================] - 2s 23ms/step - loss: 6.7617 - kullback_leibler_divergence: 6.7109 - val_loss: 6.3310 - val_kullback_leibler_divergence: 6.2595
Epoch 2/250
63/63 [==============================] - 1s 22ms/step - loss: 5.7238 - kullback_leibler_divergence: 5.6040 - val_loss: 4.8118 - val_kullback_leibler_divergence: 4.6835
Epoch 3/250
63/63 [==============================] - 1s 17ms/step - loss: 4.8752 - kullback_leibler_divergence: 4.7582 - val_loss: 4.6160 - val_kullback_leibler_divergence: 4.5028
Epoch 4/250
63/63 [==============================] - 1s 14ms/step - loss: 4.7594 - kullback_leibler_divergence: 4.6479 - val_loss: 4.5768 - val_kullback_leibler_divergence: 4.4662
Epoch 5/250
63/63 [==============================] - 1s 15ms/step - loss: 4.7055 - kullback_leibler_divergence: 4.5965 - val_loss: 4.5525 - val_kullback_leibler_divergence: 4.4445
Epoch 6/250
63/63 [==============================] - 1s 14ms/step - loss: 4.6641 - kullback_leibler_

In [13]:
vap.fit(epoch, vap.model3)

Epoch 1/250
63/63 [==============================] - 2s 17ms/step - loss: 6.8376 - kullback_leibler_divergence: 6.7936 - val_loss: 6.5404 - val_kullback_leibler_divergence: 6.4943
Epoch 2/250
63/63 [==============================] - 1s 15ms/step - loss: 6.2066 - kullback_leibler_divergence: 6.1146 - val_loss: 5.3105 - val_kullback_leibler_divergence: 5.1786
Epoch 3/250
63/63 [==============================] - 1s 15ms/step - loss: 5.3691 - kullback_leibler_divergence: 5.2456 - val_loss: 4.7572 - val_kullback_leibler_divergence: 4.6403
Epoch 4/250
63/63 [==============================] - 1s 17ms/step - loss: 5.2098 - kullback_leibler_divergence: 5.0935 - val_loss: 4.6667 - val_kullback_leibler_divergence: 4.5521
Epoch 5/250
63/63 [==============================] - 1s 17ms/step - loss: 5.0941 - kullback_leibler_divergence: 4.9807 - val_loss: 4.6352 - val_kullback_leibler_divergence: 4.5217
Epoch 6/250
63/63 [==============================] - 1s 15ms/step - loss: 5.0529 - kullback_leibler_

In [14]:
vap.fit(epoch, vap.model4)

Epoch 1/250
63/63 [==============================] - 3s 25ms/step - loss: 7.5454 - kullback_leibler_divergence: 6.7918 - val_loss: 7.2297 - val_kullback_leibler_divergence: 6.5258
Epoch 2/250
63/63 [==============================] - 1s 20ms/step - loss: 6.9432 - kullback_leibler_divergence: 6.2349 - val_loss: 6.2209 - val_kullback_leibler_divergence: 5.5112
Epoch 3/250
63/63 [==============================] - 1s 20ms/step - loss: 6.0009 - kullback_leibler_divergence: 5.3229 - val_loss: 5.3034 - val_kullback_leibler_divergence: 4.6630
Epoch 4/250
63/63 [==============================] - 1s 19ms/step - loss: 5.6588 - kullback_leibler_divergence: 5.0593 - val_loss: 5.1475 - val_kullback_leibler_divergence: 4.5841
Epoch 5/250
63/63 [==============================] - 1s 19ms/step - loss: 5.5179 - kullback_leibler_divergence: 4.9801 - val_loss: 5.0385 - val_kullback_leibler_divergence: 4.5331
Epoch 6/250
63/63 [==============================] - 1s 19ms/step - loss: 5.4236 - kullback_leibler_

In [15]:
vap.fit(epoch, vap.model5)

Epoch 1/250
63/63 [==============================] - 3s 24ms/step - loss: 7.8938 - kullback_leibler_divergence: 7.1803 - val_loss: 7.6055 - val_kullback_leibler_divergence: 6.9366
Epoch 2/250
63/63 [==============================] - 1s 20ms/step - loss: 7.1938 - kullback_leibler_divergence: 6.5082 - val_loss: 6.1867 - val_kullback_leibler_divergence: 5.5040
Epoch 3/250
63/63 [==============================] - 1s 19ms/step - loss: 6.1778 - kullback_leibler_divergence: 5.5250 - val_loss: 5.5579 - val_kullback_leibler_divergence: 4.9298
Epoch 4/250
63/63 [==============================] - 1s 19ms/step - loss: 5.9399 - kullback_leibler_divergence: 5.3471 - val_loss: 5.3609 - val_kullback_leibler_divergence: 4.7983
Epoch 5/250
63/63 [==============================] - 1s 19ms/step - loss: 5.7442 - kullback_leibler_divergence: 5.2057 - val_loss: 5.2472 - val_kullback_leibler_divergence: 4.7387
Epoch 6/250
63/63 [==============================] - 1s 19ms/step - loss: 5.6402 - kullback_leibler_

In [21]:
vap.fit(epoch, vap.model6)

Epoch 1/250
63/63 [==============================] - 3s 29ms/step - loss: 6.6879 - kullback_leibler_divergence: 6.4709 - val_loss: 5.3119 - val_kullback_leibler_divergence: 5.0825
Epoch 2/250
63/63 [==============================] - 1s 23ms/step - loss: 5.3001 - kullback_leibler_divergence: 5.1140 - val_loss: 4.7131 - val_kullback_leibler_divergence: 4.5475
Epoch 3/250
63/63 [==============================] - 2s 26ms/step - loss: 5.0867 - kullback_leibler_divergence: 4.9234 - val_loss: 4.6502 - val_kullback_leibler_divergence: 4.4883
Epoch 4/250
63/63 [==============================] - 1s 23ms/step - loss: 5.0226 - kullback_leibler_divergence: 4.8602 - val_loss: 4.6341 - val_kullback_leibler_divergence: 4.4698
Epoch 5/250
63/63 [==============================] - 1s 23ms/step - loss: 4.9670 - kullback_leibler_divergence: 4.8017 - val_loss: 4.6165 - val_kullback_leibler_divergence: 4.4484
Epoch 6/250
63/63 [==============================] - 1s 23ms/step - loss: 4.9305 - kullback_leibler_

### 

### Performance of Models in amalgam_predict

In [16]:
import amalgam_predict as ap

data = ap.load_data()

In [17]:
ap.fit(epoch, ap.model6(data), data)

Epoch 1/250
63/63 [==============================] - 2s 23ms/step - loss: 5.6824 - kullback_leibler_divergence: 5.3791 - val_loss: 4.9645 - val_kullback_leibler_divergence: 4.5653
Epoch 2/250
63/63 [==============================] - 1s 19ms/step - loss: 5.3579 - kullback_leibler_divergence: 4.8983 - val_loss: 5.0198 - val_kullback_leibler_divergence: 4.5216
Epoch 3/250
63/63 [==============================] - 1s 19ms/step - loss: 5.3349 - kullback_leibler_divergence: 4.8115 - val_loss: 4.9639 - val_kullback_leibler_divergence: 4.4254
Epoch 4/250
63/63 [==============================] - 1s 19ms/step - loss: 5.2868 - kullback_leibler_divergence: 4.7088 - val_loss: 4.9132 - val_kullback_leibler_divergence: 4.3221
Epoch 5/250
63/63 [==============================] - 1s 20ms/step - loss: 5.2370 - kullback_leibler_divergence: 4.6297 - val_loss: 4.9226 - val_kullback_leibler_divergence: 4.2974
Epoch 6/250
63/63 [==============================] - 1s 20ms/step - loss: 5.2149 - kullback_leibler_

In [18]:
ap.fit(epoch, ap.model7(data), data)

Epoch 1/250
63/63 [==============================] - 3s 27ms/step - loss: 6.8141 - kullback_leibler_divergence: 6.7028 - val_loss: 5.5546 - val_kullback_leibler_divergence: 5.4121
Epoch 2/250
63/63 [==============================] - 1s 24ms/step - loss: 5.4686 - kullback_leibler_divergence: 5.3484 - val_loss: 4.9300 - val_kullback_leibler_divergence: 4.8175
Epoch 3/250
63/63 [==============================] - 2s 24ms/step - loss: 5.2603 - kullback_leibler_divergence: 5.1462 - val_loss: 4.8453 - val_kullback_leibler_divergence: 4.7272
Epoch 4/250
63/63 [==============================] - 1s 23ms/step - loss: 5.1657 - kullback_leibler_divergence: 5.0456 - val_loss: 4.8144 - val_kullback_leibler_divergence: 4.6909
Epoch 5/250
63/63 [==============================] - 1s 24ms/step - loss: 5.1064 - kullback_leibler_divergence: 4.9803 - val_loss: 4.7785 - val_kullback_leibler_divergence: 4.6516
Epoch 6/250
63/63 [==============================] - 2s 24ms/step - loss: 5.0653 - kullback_leibler_

In [19]:
ap.fit(epoch, ap.model8(data), data)

Epoch 1/250
63/63 [==============================] - 3s 28ms/step - loss: 6.6214 - kullback_leibler_divergence: 6.3911 - val_loss: 4.9503 - val_kullback_leibler_divergence: 4.7217
Epoch 2/250
63/63 [==============================] - 2s 25ms/step - loss: 5.3405 - kullback_leibler_divergence: 5.1475 - val_loss: 4.7250 - val_kullback_leibler_divergence: 4.5377
Epoch 3/250
63/63 [==============================] - 2s 25ms/step - loss: 5.1581 - kullback_leibler_divergence: 4.9750 - val_loss: 4.6827 - val_kullback_leibler_divergence: 4.4973
Epoch 4/250
63/63 [==============================] - 2s 26ms/step - loss: 5.0868 - kullback_leibler_divergence: 4.9035 - val_loss: 4.6606 - val_kullback_leibler_divergence: 4.4740
Epoch 5/250
63/63 [==============================] - 2s 28ms/step - loss: 5.0232 - kullback_leibler_divergence: 4.8355 - val_loss: 4.6263 - val_kullback_leibler_divergence: 4.4356
Epoch 6/250
63/63 [==============================] - 2s 25ms/step - loss: 4.9764 - kullback_leibler_

In [20]:
ap.fit(epoch, ap.model9(data), data)

Epoch 1/250
63/63 [==============================] - 3s 31ms/step - loss: 6.8182 - kullback_leibler_divergence: 6.6854 - val_loss: 5.2245 - val_kullback_leibler_divergence: 5.0482
Epoch 2/250
63/63 [==============================] - 2s 24ms/step - loss: 5.5170 - kullback_leibler_divergence: 5.3820 - val_loss: 4.9151 - val_kullback_leibler_divergence: 4.7878
Epoch 3/250
63/63 [==============================] - 2s 24ms/step - loss: 5.3082 - kullback_leibler_divergence: 5.1817 - val_loss: 4.8407 - val_kullback_leibler_divergence: 4.7160
Epoch 4/250
63/63 [==============================] - 2s 25ms/step - loss: 5.2148 - kullback_leibler_divergence: 5.0837 - val_loss: 4.8064 - val_kullback_leibler_divergence: 4.6734
Epoch 5/250
63/63 [==============================] - 2s 25ms/step - loss: 5.1258 - kullback_leibler_divergence: 4.9903 - val_loss: 4.7858 - val_kullback_leibler_divergence: 4.6500
Epoch 6/250
63/63 [==============================] - 2s 25ms/step - loss: 5.0785 - kullback_leibler_